# Preprocessing of data before feeding it to JASP

In [6]:
# Load data
import pandas as pd
data = pd.read_csv('data.csv')

In [7]:
# Remove failed attention checks
to_drop = []
for index, row in data.iterrows():
    if row['attention_check_name'] != 4 and row['attention_check_topic'] != 1:
        to_drop.append(index)
        continue
    if row['own_personality_q5'] != 6 and row['rosas_q7'] != 0 and row['personality_q14'] != 6:
        to_drop.append(index)
        
print(f'Dropped {len(to_drop)} rows')
data = data.drop(to_drop, axis=0)
data

Dropped 6 rows


,age,gender,country,english,condition,attention_check_name,attention_check_topic,own_personality_q1,own_personality_q2,own_personality_q3,...,rosas_q4,rosas_q5,rosas_q6,rosas_q7,rosas_q8,rosas_q9,rosas_q10,rosas_q11,rosas_q12,rosas_q13
0,19,female,ZA,high,strap_ext_yes,2,1,4,2,4,...,0,1,0,0,3,3,3,2,3,3
1,32,female,ZA,high,strap_int_yes,4,1,4,4,5,...,1,1,1,0,2,3,3,2,2,2
2,22,female,EE,med,gpt_int_yes,4,1,5,4,4,...,2,4,2,0,0,1,0,0,0,0
3,23,female,PT,med,gpt_ext_yes,2,1,5,3,4,...,0,3,1,0,1,2,1,2,2,1
4,39,male,CA,high,strap_ext_yes,2,1,4,4,5,...,2,2,1,0,2,4,4,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,30,male,PL,med,gpt_int_yes,4,1,3,1,1,...,0,2,2,0,3,3,3,2,1,0
76,30,male,ZA,high,gpt_ext_yes,4,1,0,0,0,...,3,4,3,0,2,0,0,0,0,0
77,33,male,IT,med,strap_ext_yes,0,1,3,4,4,...,2,3,1,0,2,3,2,1,1,0
78,24,male,ZA,med,gpt_int_yes,0,1,6,6,0,...,4,4,4,0,0,0,0,0,0,0


In [8]:
# Group measures
data2jasp = pd.DataFrame()
data2jasp[['Age', 'Gender', 'English']] = data[['age', 'gender', 'english']]
data2jasp['Condition'] = data['condition'].apply(lambda x: x[:-4])
data2jasp['Model'] = data['condition'].apply(lambda x: "GPT-3" if x.split("_")[0] == "gpt" else "STRAP")
data2jasp['Personality condition'] = data['condition'].apply(lambda x: "Ext" if x.split("_")[1] == "ext" else "Int")
data2jasp['Own introversion'] = data[[f'own_personality_q{i+1}' for i in range(4)]].mean(axis=1)
data2jasp['Own extraversion'] = data[[f'own_personality_q{i+6}' for i in range(4)]].mean(axis=1)
data2jasp['Own personality'] = 'Ext'
data2jasp['Own personality'][data2jasp['Own extraversion'] < data2jasp['Own introversion']] = 'Int'
data2jasp['Personality matching'] = data2jasp['Own personality'] == data2jasp['Personality condition']
data2jasp['Introversion rating'] = data[[f'personality_q{i+1}' for i in range(5)]].mean(axis=1)
data2jasp['Extraversion rating'] = data[[f'personality_q{i+6}' for i in range(5)]].mean(axis=1)
data2jasp['Extraversion'] = data2jasp['Extraversion rating'] + 6 - data2jasp['Introversion rating']
data2jasp['Introversion'] = data2jasp['Introversion rating'] + 6 - data2jasp['Extraversion rating']
data2jasp['Fluency rating'] = data[[f'personality_q{i+11}' for i in range(3)]].mean(axis=1)
data2jasp['Godspeed anthropomorphism'] = data[[f'godspeed_q{i+1}' for i in range(5)]].mean(axis=1)
data2jasp['Godspeed likeability'] = data[[f'godspeed_q{i+6}' for i in range(5)]].mean(axis=1)
data2jasp['Godspeed safety'] = data[[f'godspeed_q{i+11}' for i in range(3)]].mean(axis=1)
data2jasp['RoSAS warmth'] = data[[f'rosas_q{i+1}' for i in range(6)]].mean(axis=1)
data2jasp['RoSAS discomfort'] = data[[f'rosas_q{i+8}' for i in range(6)]].mean(axis=1)
data2jasp['Uncanniness'] = (data2jasp['RoSAS discomfort'] - data2jasp['RoSAS warmth'] - data2jasp['Godspeed safety'] - data2jasp['Godspeed anthropomorphism'] + 15) / 4
overall = []
for i, row in data2jasp.iterrows():
    score = row['Extraversion rating'] - row['Introversion rating']
    if row['Personality condition'] == 'int':
        score *= -1
    score += 6
    score /= 2
    overall.append(score)
data2jasp['Personality overall'] = overall
data2jasp.to_csv('data2jasp.csv', index=False)
data2jasp

/home/alessio/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Age,Gender,English,Condition,Model,Personality condition,Own introversion,Own extraversion,Own personality,Personality matching,...,Extraversion,Introversion,Fluency rating,Godspeed anthropomorphism,Godspeed likeability,Godspeed safety,RoSAS warmth,RoSAS discomfort,Uncanniness,Personality overall
0,19,female,high,strap_ext,STRAP,Ext,3.00,3.75,Ext,True,...,11.2,0.8,6.000000,1.8,2.8,3.000000,1.333333,2.833333,2.925000,5.6
1,32,female,high,strap_int,STRAP,Int,3.75,2.75,Int,True,...,7.2,4.8,2.666667,2.0,3.0,3.333333,1.333333,2.333333,2.666667,3.6
2,22,female,med,gpt_int,GPT-3,Int,4.00,2.00,Int,True,...,6.2,5.8,5.666667,4.0,5.0,2.333333,2.833333,0.166667,1.500000,3.1
3,23,female,med,gpt_ext,GPT-3,Ext,3.25,3.75,Ext,True,...,8.8,3.2,4.333333,3.0,3.4,2.333333,1.666667,1.500000,2.375000,4.4
4,39,male,high,strap_ext,STRAP,Ext,4.50,3.25,Int,False,...,9.0,3.0,5.000000,2.6,3.4,2.666667,2.166667,2.000000,2.391667,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,30,male,med,gpt_int,GPT-3,Int,2.00,3.75,Ext,False,...,6.8,5.2,6.000000,2.2,3.4,3.333333,1.333333,2.000000,2.533333,3.4
76,30,male,high,gpt_ext,GPT-3,Ext,0.00,5.00,Ext,True,...,5.2,6.8,6.000000,3.6,4.8,2.666667,3.333333,0.333333,1.433333,2.6
77,33,male,med,strap_ext,STRAP,Ext,3.75,3.25,Int,False,...,9.6,2.4,5.333333,2.0,3.6,2.666667,2.666667,1.500000,2.291667,4.8
78,24,male,med,gpt_int,GPT-3,Int,3.00,3.25,Ext,False,...,9.0,3.0,5.666667,3.8,5.0,2.333333,4.000000,0.000000,1.216667,4.5
